In [1]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://localhost:9200")
client.info()
client.ping()

True

In [2]:
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})


In [3]:
print("Got %d Hits:" % resp['hits']['total'])

Got 22329 Hits:


In [4]:
import pandas as pd

In [15]:
df_eng = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])
df_de = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])

In [23]:
df_eng.head()

,name,sdesc,desc,category,categorypath


In [5]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            name_en.append(name["value"])
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            sdesc_en.append(sdesc["value"])
        else:
            sdesc_de.append(sdesc["value"])

    for desc in hit['_source']['descriptionsSource']:
        if (desc["language"]=="en"):
            desc_en.append(desc["value"])
        else:
            desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            cat_en.append(cats["label"])
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            catpath_en.append(cats["path"])
        else:
            catpath_de.append(cats["path"])


In [8]:
df_en = pd.DataFrame({'name':name_en,
                       'sdesc':sdesc_en,
                       'desc':desc_en,
                       'category':cat_en,
                       'categorypath':catpath_en})


# df_de = pd.DataFrame({'name':name_de,
#                        'sdesc':sdesc_de,
#                        'desc':desc_de,
#                        'category':cat_de,
#                        'categorypath':catpath_de})

In [9]:
df_en.head()

,name,sdesc,desc,category,categorypath
0,"DENSO DTM82363 Thermostat, coolant","Thermostat, coolant",<div class='criterias'><table class='criterias...,Thermostat,SparePart/Cooling System/Thermostat
1,ZF 1043.010.297 Automatic Transmission,Automatic Transmission,<div class='criterias'><table class='criterias...,Automatic Transmission,SparePart/Transmission/Automatic Transmission/...
2,ZF 1060.040.003 Automatic Transmission,Automatic Transmission,<div class='criterias'><table class='criterias...,Automatic Transmission,SparePart/Transmission/Automatic Transmission/...
3,VAICO V10-0131 Camshaft,Camshaft,<div class='criterias'><table class='criterias...,Camshaft,SparePart/Engine/Engine Timing/Camshaft
4,VAICO V26-0302 Track Control Arm,Track Control Arm,<div class='criterias'><table class='criterias...,Track Control Arm,SparePart/Powertrain & Suspension/Wheel Suspen...
